# Generate Test Dataset

증권사 리포트를 파싱한 데이터를 기반으로 RAG의 성능을 측정하기 위한 테스트 데이터셋 생성을 목표

Openai GPT의 도움을 받아 생성하는 방법 사용

In [22]:
import os, json
from dotenv import load_dotenv

import joblib

import pandas as pd
from openai import OpenAI # openai==1.52.2
import traceback

In [4]:
load_dotenv() ## OPEN_API_KEY 속성이 존재해야함

True

In [ ]:
# 2. chunk된 텍스트 데이터를 담은 배열 예시
chunked_texts = [
    "이 텍스트는 인공지능의 역사와 발전 과정에 대해 설명합니다. 초기 AI 연구부터 최신 딥러닝 기술까지 다양한 사례를 다룹니다.",
    "이 문서는 데이터 전처리의 중요성과 여러 전처리 기법들을 설명하며, 실제 예시와 코드 스니펫을 포함하고 있습니다."
]


In [17]:
client = OpenAI()


In [88]:
def generate_qa_pairs(text_chunk):
    """
    주어진 json list 형식의 청크 리스트트에 대해 질답 쌍과 관련성 점수를 생성하는 프롬프트를 구성하여
    ChatGPT API를 호출한 후 결과를 반환합니다.
    
    출력은 JSON 형식의 문자열로, 각 질답 쌍에는 question, answer, document_id 항목이 포함됩니다.
    """

    prompt = f"""
    금융 도메인에 대한 QA 쌍을 생성하는 것이 너의 목표야.

    아래에 주어진 청크 리스트를 읽고, 각 청크의 "page_content" 내용을 기반으로 QA 쌍을 생성해줘.
    QA 쌍은 아래 세 가지 질문 유형 각각에 대해 1문제씩 만들어야 해.
    질문은 해당 청크의 정보를 기반으로 작성하고, 답변은 아래 출력 형식을 준수하는 JSON 객체로 만들어줘.
    또한, 생성된 QA 쌍마다 생성했을 때 참조한 문서의 ID를 "answer_document_id" 항목에 포함시켜줘.

    질문 유형:
    1. 상식적 질문 (General Knowledge Questions)
    - 예: "애플(Apple)의 창립 연도는?"
    2. 수치 기반 질문 (Numeric Questions)
    - 예: "2022년 애플의 총수익은 얼마인가?"
    3. 추론 질문 (Inference Questions)
    - 예: "애플의 순이익이 작년 대비 증가했는가?"

    출력 형식:
    {{
    "qa_pairs": [
        {{
        "question": "질문 내용",
        "answer": "답변 내용",
        "answer_document_id": 123  // 정답 컨텍스트의 id (숫자)
        }},
        ...
    ]
    }}

    청크 리스트:
    {text_chunk}
        """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 실제 사용 모델 이름으로 변경
            messages=[
                {"role": "system", "content": "너는 금융 도메인의 전문가야."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_completion_tokens=500,
            timeout=60
        )
        
        # API 응답에서 메시지 내용 추출
        print("----Response:", response)
        content = response.choices[0].message.content
        print("API 응답:", content)
        # JSON 파싱 시도 (모델이 형식을 잘 지켰다면 파싱 가능)
        result = json.loads(content)
        return result
    except Exception as e:
        print("API 호출 중 오류 발생:", e)
        traceback.print_exc()
        return None


In [5]:
chunks = joblib.load('../data/final_documents.pkl')

In [69]:
print("청크 개수:", len(chunks))
print("첫 번째 청크 예시:", chunks[0])


청크 개수: 363
첫 번째 청크 예시: page_content='20240509에 미래에셋증권에서 발행한 크래프톤에 관한 레포트에서 나온 내용.

Section Title: # 24년에도 질주는 계속된다

1Q24 Review: 컨센서스 상회' metadata={'category': 'paragraph', 'coordinates': [{'x': 0.355, 'y': 0.1922}, {'x': 0.5885, 'y': 0.1922}, {'x': 0.5885, 'y': 0.2094}, {'x': 0.355, 'y': 0.2094}], 'page': 1, 'id': 6, 'company_name': '크래프톤', 'report_date': '20240509', 'securities_firm': '미래에셋증권', 'source_file': '크래프톤_20240509_미래에셋증권.pdf', 'content_types': {'markdown': '1Q24 Review: 컨센서스 상회', 'text': '1Q24 Review: 컨센서스 상회', 'html': "<p id='6' data-category='paragraph' style='font-size:18px'>1Q24 Review: 컨센서스 상회</p>"}, 'uuid': '81bb64f6-cff5-4591-acc3-013c90a1d10b', 'nid': 2}


In [8]:
partial_chunks = {}
for chunk in chunks:
  text = chunk.page_content
  metadata = chunk.metadata
  
  company_name = metadata['company_name']
  if company_name in partial_chunks:
    partial_chunks[company_name].append(chunk)
  else :
    partial_chunks[company_name] = []
    

    

In [89]:
error_num = 0
idx = 0
for company_name, chunks in partial_chunks.items():
  print(f"\n--- {company_name} ---")
  if idx == 1 : break
  
  file_dict = {}
  for chunk in chunks:
    dict_chunk = {}
    # dict_chunk['metadata'] = chunk.metadata
    dict_chunk['page_content'] = chunk.page_content
    dict_chunk['answer_document_id'] = chunk.metadata['nid']
    dict_chunk['source_file'] = chunk.metadata['source_file']
    dict_chunk['metadata'] = chunk.metadata
    if file_dict.get(chunk.metadata['source_file']) is None :
        file_dict[chunk.metadata['source_file']] = [dict_chunk]
    else :
        file_dict[chunk.metadata['source_file']].append(dict_chunk)
    
  for source_file, chunks in file_dict.items():
    chunk_list = []
    for chunk in chunks:
      dict_chunk = {}
      # dict_chunk['metadata'] = chunk['metadata']
      dict_chunk['page_content'] = chunk['page_content']
      dict_chunk['answer_document_id'] = chunk['metadata']['nid']
      chunk_list.append(dict_chunk)
    
    print("chunk list len", len(chunk_list))
    # for i in chunk_list:
    #     print("len of chunk",len(json.dumps(i)))
    # print("len of chunk",len(json.dumps(chunk_list[0])))
    json_chunk = json.dumps(chunk_list, ensure_ascii=False)
    # print(json_chunk)
      
    qa_result = generate_qa_pairs(json_chunk)
    if qa_result is not None:
        print(json.dumps(qa_result, indent=2, ensure_ascii=False))
    else:
        print("결과 생성 실패")
        error_num += 1
        
  idx += 1



--- 크래프톤 ---
chunk list len 27
----Response: ChatCompletion(id='chatcmpl-AweTIQk4xHSsjG2kiJ8ORhofJ7cHE', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "qa_pairs": [\n    // General Knowledge Questions\n    {\n      "question": "크래프톤의 2024년 목표주가는 얼마인가?",\n      "answer": "370,000원입니다.",\n      "answer_document_id": 6\n    },\n    {\n      "question": "미래에셋증권에서 크래프톤에 관한 레포트가 발행된 날짜는?",\n      "answer": "2024년 5월 9일입니다.",\n      "answer_document_id": 2\n    },\n    {\n      "question": "다크앤다커M의 정식 출시 목표는 언제인가?",\n      "answer": "2024년 4분기입니다.",\n      "answer_document_id": 4\n    },\n    \n    // Numeric Questions\n    {\n      "question": "2024년 크래프톤의 예상 매출액은 얼마인가?",\n      "answer": "2,192억원입니다.",\n      "answer_document_id": 7\n    },\n    {\n      "question": "2023년 크래프톤의 영업이익은 얼마인가?",\n      "answer": "768억원입니다.",\n      "answer_document_id": 8\n    },\n    {\n      "question": "1분기 2024년 크래프톤의 매출액은 얼마인가?",\n   

Traceback (most recent call last):
  File "C:\Users\jilp1\AppData\Local\Temp\ipykernel_10244\2018625126.py", line 58, in generate_qa_pairs
    result = json.loads(content)
             ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


----Response: ChatCompletion(id='chatcmpl-AweTUg7CnCnt1tjLW1twwESzsieka', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "qa_pairs": [\n    // 상식적 질문\n    {\n      "question": "크래프톤의 목표주가는 얼마인가?",\n      "answer": "410,000 원입니다.",\n      "answer_document_id": 94\n    },\n    {\n      "question": "크래프톤의 상장 연도는 언제인가?",\n      "answer": "크래프톤은 2021년에 상장되었습니다.",\n      "answer_document_id": 95\n    },\n    {\n      "question": "크래프톤의 발행주식수는 얼마인가?",\n      "answer": "4,790 만주입니다.",\n      "answer_document_id": 95\n    },\n    \n    // 수치 기반 질문\n    {\n      "question": "2024년 크래프톤의 예상 매출액은 얼마인가?",\n      "answer": "2,685억원입니다.",\n      "answer_document_id": 104\n    },\n    {\n      "question": "2023년 3분기 크래프톤의 영업이익은 얼마인가?",\n      "answer": "2,404억원입니다.",\n      "answer_document_id": 99\n    },\n    {\n      "question": "크래프톤의 52주 최고가는 얼마인가?",\n      "answer": "349,000 원입니다.",\n      "answer_document_id": 95\n    },\n    \

Traceback (most recent call last):
  File "C:\Users\jilp1\AppData\Local\Temp\ipykernel_10244\2018625126.py", line 58, in generate_qa_pairs
    result = json.loads(content)
             ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


----Response: ChatCompletion(id='chatcmpl-AweTbfIuxwyHglnX8vDJu8zEHk3nE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "qa_pairs": [\n    // General Knowledge Questions\n    {\n      "question": "크래프톤의 목표주가는 얼마인가?",\n      "answer": "490,000원입니다.",\n      "answer_document_id": 412\n    },\n    {\n      "question": "2024년 크래프톤의 예상 매출액은 얼마인가?",\n      "answer": "2조 7,075억원입니다.",\n      "answer_document_id": 421\n    },\n    {\n      "question": "크래프톤의 시가총액은 얼마인가?",\n      "answer": "15,614.4억원입니다.",\n      "answer_document_id": 413\n    },\n    \n    // Numeric Questions\n    {\n      "question": "2023년 크래프톤의 영업이익은 얼마인가?",\n      "answer": "768.0억원입니다.",\n      "answer_document_id": 416\n    },\n    {\n      "question": "크래프톤의 현재 주가는 얼마인가?",\n      "answer": "326,000원입니다.",\n      "answer_document_id": 412\n    },\n    {\n      "question": "2024년 크래프톤의 순이익 예상치는 얼마인가?",\n      "answer": "971.2억원입니다.",\n      "answer_doc

...

----Response: ChatCompletion(id='chatcmpl-AweU6leCvKCmCBpMaUYcymxRUFP6a', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "qa_pairs": [\n    {\n      "question": "크래프톤의 1Q24 매출액은 얼마인가?",\n      "answer": "크래프톤의 1Q24 매출액은 6,659억원이다.",\n      "answer_document_id": 2669\n    },\n    {\n      "question": "크래프톤의 2024년 목표주가는 얼마인가?",\n      "answer": "크래프톤의 2024년 목표주가는 32만원이다.",\n      "answer_document_id": 2674\n    },\n    {\n      "question": "크래프톤의 지배순이익은 작년 대비 얼마 증가했는가?",\n      "answer": "크래프톤의 지배순이익은 작년 대비 30.9% 증가했다.",\n      "answer_document_id": 2669\n    },\n    {\n      "question": "크래프톤의 2분기 PC 매출액은 얼마인가?",\n      "answer": "크래프톤의 2분기 PC 매출액은 2,437억원이다.",\n      "answer_document_id": 2670\n    },\n    {\n      "question": "2022년 크래프톤의 영업이익은 얼마였는가?",\n      "answer": "2022년 크래프톤의 영업이익은 752억원이었다.",\n      "answer_document_id": 2682\n    },\n    {\n      "question": "크래프톤의 모바일 매출액이 작년 대비 얼마나 증가했는가?",\n      "answer

Traceback (most recent call last):
  File "C:\Users\jilp1\AppData\Local\Temp\ipykernel_10244\2018625126.py", line 58, in generate_qa_pairs
    result = json.loads(content)
             ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


----Response: ChatCompletion(id='chatcmpl-AweUGLqTI5NI0eQdIfXfYPg90YWAl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "qa_pairs": [\n    // General Knowledge Questions\n    {\n      "question": "20241018에 SK증권에서 발행한 크래프톤의 목표주가는 얼마인가?",\n      "answer": "420,000 원입니다.",\n      "answer_document_id": 2863\n    },\n    {\n      "question": "크래프톤의 발행주식수는 얼마인가?",\n      "answer": "4,790 만주입니다.",\n      "answer_document_id": 2864\n    },\n    {\n      "question": "크래프톤의 주요주주는 누구인가?",\n      "answer": "장병규(외31)와 IMAGE FRAME INVESTMENT(HK)입니다.",\n      "answer_document_id": 2864\n    },\n    \n    // Numeric Questions\n    {\n      "question": "크래프톤의 현재주가는 얼마인가?",\n      "answer": "334,500 원입니다.",\n      "answer_document_id": 2863\n    },\n    {\n      "question": "3Q24 매출액은 얼마로 예상되나요?",\n      "answer": "6,737 억원입니다.",\n      "answer_document_id": 2867\n    },\n    {\n      "question": "2024년 크래프톤의 영업이익 예상치는 얼마인가?",\n      

Traceback (most recent call last):
  File "C:\Users\jilp1\AppData\Local\Temp\ipykernel_10244\2018625126.py", line 58, in generate_qa_pairs
    result = json.loads(content)
             ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jilp1\.pyenv\pyenv-win\versions\3.11.9\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 